# Torch Neural Network Integration with custom integration function

## Libraries imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import math

from mpmath import polylog, exp

## Model definition

### Neural Network implementation

In [2]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Sigmoid()
        self.output_layer = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x

In [3]:
def train_model(model, criterion, optimizer, x_train, y_train, epochs):
    for epoch in range(epochs):
        predictions = model(x_train)
        loss = criterion(predictions, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.6f}')

### NN custom integration function

In [4]:
def get_NN_analytical_integral(alpha, beta, b1, b2, weights_w1, weights_w2):

    def compute_phi_j(b1_j, w1_j, alpha, beta):
        term_alpha = polylog(1, -exp(-b1_j - w1_j * alpha))
        term_beta = polylog(1, -exp(-b1_j - w1_j * beta))
        return term_alpha - term_beta

    integral_sum = 0
    for w2, w1, b1_j in zip(weights_w2, weights_w1, b1):
        phi_j = compute_phi_j(b1_j, w1, alpha, beta)
        integral_sum += w2 * ((beta - alpha) + phi_j / w1)
    
    return b2 * (beta - alpha) + integral_sum

### Dataset generator

In [5]:
def generate_data(func, n_samples=100):
    x = torch.linspace(0, 1, n_samples).unsqueeze(1)
    y = func(x)
    return x, y

def cos_func(x):
    return torch.cos(x)

def exp_func(x):
    return torch.exp(x)

## Model application

### Hyperparams

In [11]:
input_size = 1
hidden_size = 10
learning_rate = 0.01
num_epochs = 10000

model_cos = MLP(input_size, hidden_size)
model_exp = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_cos = optim.Adam(model_cos.parameters(), lr=learning_rate)
optimizer_exp = optim.Adam(model_exp.parameters(), lr=learning_rate)

### Dataset generation

In [12]:
x_train_cos, y_train_cos = generate_data(cos_func, 10000)
x_train_exp, y_train_exp = generate_data(exp_func, 10000)

### Model training

In [13]:
train_model(model_cos, criterion, optimizer_cos, x_train_cos, y_train_cos, num_epochs)

Epoch [100/10000], Loss: 0.017409
Epoch [200/10000], Loss: 0.005855
Epoch [300/10000], Loss: 0.001881
Epoch [400/10000], Loss: 0.001346
Epoch [500/10000], Loss: 0.001309
Epoch [600/10000], Loss: 0.001294
Epoch [700/10000], Loss: 0.001278
Epoch [800/10000], Loss: 0.001262
Epoch [900/10000], Loss: 0.001244
Epoch [1000/10000], Loss: 0.001226
Epoch [1100/10000], Loss: 0.001207
Epoch [1200/10000], Loss: 0.001188
Epoch [1300/10000], Loss: 0.001167
Epoch [1400/10000], Loss: 0.001146
Epoch [1500/10000], Loss: 0.001123
Epoch [1600/10000], Loss: 0.001099
Epoch [1700/10000], Loss: 0.001073
Epoch [1800/10000], Loss: 0.001044
Epoch [1900/10000], Loss: 0.001013
Epoch [2000/10000], Loss: 0.000979
Epoch [2100/10000], Loss: 0.000940
Epoch [2200/10000], Loss: 0.000897
Epoch [2300/10000], Loss: 0.000847
Epoch [2400/10000], Loss: 0.000791
Epoch [2500/10000], Loss: 0.000727
Epoch [2600/10000], Loss: 0.000654
Epoch [2700/10000], Loss: 0.000573
Epoch [2800/10000], Loss: 0.000486
Epoch [2900/10000], Loss: 0.0

In [14]:
train_model(model_exp, criterion, optimizer_exp, x_train_exp, y_train_exp, num_epochs)

Epoch [100/10000], Loss: 0.169339
Epoch [200/10000], Loss: 0.100404
Epoch [300/10000], Loss: 0.045425
Epoch [400/10000], Loss: 0.015194
Epoch [500/10000], Loss: 0.006290
Epoch [600/10000], Loss: 0.004557
Epoch [700/10000], Loss: 0.003917
Epoch [800/10000], Loss: 0.003441
Epoch [900/10000], Loss: 0.003038
Epoch [1000/10000], Loss: 0.002680
Epoch [1100/10000], Loss: 0.002352
Epoch [1200/10000], Loss: 0.002047
Epoch [1300/10000], Loss: 0.001763
Epoch [1400/10000], Loss: 0.001499
Epoch [1500/10000], Loss: 0.001256
Epoch [1600/10000], Loss: 0.001035
Epoch [1700/10000], Loss: 0.000839
Epoch [1800/10000], Loss: 0.000667
Epoch [1900/10000], Loss: 0.000521
Epoch [2000/10000], Loss: 0.000400
Epoch [2100/10000], Loss: 0.000302
Epoch [2200/10000], Loss: 0.000225
Epoch [2300/10000], Loss: 0.000167
Epoch [2400/10000], Loss: 0.000124
Epoch [2500/10000], Loss: 0.000093
Epoch [2600/10000], Loss: 0.000071
Epoch [2700/10000], Loss: 0.000056
Epoch [2800/10000], Loss: 0.000045
Epoch [2900/10000], Loss: 0.0

### Extractoin of the parameters of the trained model

In [15]:
b1_cos = model_cos.input_layer.bias.detach().numpy()
weights_w1_cos = model_cos.input_layer.weight.detach().numpy().flatten()
b2_cos = model_cos.output_layer.bias.item()
weights_w2_cos = model_cos.output_layer.weight.detach().numpy().flatten()

In [16]:
b1_exp = model_exp.input_layer.bias.detach().numpy()
weights_w1_exp = model_exp.input_layer.weight.detach().numpy().flatten()
b2_exp = model_exp.output_layer.bias.item()
weights_w2_exp = model_exp.output_layer.weight.detach().numpy().flatten()

### Integreation parameters

In [17]:
alpha = 0
beta = 1

### NNI integral calculation

In [18]:
integral_cos = get_NN_analytical_integral(alpha, beta, b1_cos, b2_cos, weights_w1_cos, weights_w2_cos)

In [19]:
print(f"The analytical integral of cos(x) of the trained neural network model is: {float(integral_cos):.6f}")

The analytical integral of cos(x) of the trained neural network model is: 0.841521


In [20]:
integral_exp = get_NN_analytical_integral(alpha, beta, b1_exp, b2_exp, weights_w1_exp, weights_w2_exp)

In [21]:
print(f"The analytical integral of exp(x) of the trained neural network model is: {float(integral_exp):.6f}")

The analytical integral of exp(x) of the trained neural network model is: 1.718553


### True integral for comparison

$\int_{\alpha}^{\beta} dx\{cos(x)\} = sin(\beta) - sin(\alpha) $

$\int_{\alpha}^{\beta} dx\{exp(x)\} = exp(\beta) - exp(\alpha) $

In [22]:
true_integral_cos = np.sin(beta) - np.sin(alpha)
true_integral_exp = np.exp(beta) - np.exp(alpha)

In [23]:
print(f"The true cosinus integral is: {float(true_integral_cos):.6f}")

The true cosinus integral is: 0.841471


In [24]:
print(f"The true exponent integral is: {float(true_integral_exp):.6f}")

The true exponent integral is: 1.718282
